In [2]:
import pandas as pd 

In [3]:
df=pd.read_csv("C:\\Users\\DELL\\Downloads\\100_Unique_QA_Dataset.csv")

In [4]:
df.shape

(90, 2)

In [5]:
df

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100
...,...,...
85,Who directed the movie 'Titanic'?,JamesCameron
86,Which superhero is also known as the Dark Knight?,Batman
87,What is the capital of Brazil?,Brasilia
88,Which fruit is known as the king of fruits?,Mango


In [6]:
def tokenize(text):
    text=text.lower()
    text=text.replace("?","")
    text=text.replace(".","")
    return text.split()

In [7]:
df["question"][0]

'What is the capital of France?'

In [8]:
tokenize("What is the capital of France?")

['what', 'is', 'the', 'capital', 'of', 'france']

## tokenization is done now lets make our vocab 


In [9]:
vocab={"unk":0}
def build_vocab(row):
    print(row["question"])
    print(row["answer"])
    tokens_questions=tokenize(row["question"])
    tokens_answer=tokenize(row["answer"])
    merged_texts=tokens_questions+tokens_answer

    for text in (merged_texts):
        if text not in vocab:
            vocab[text]=len(vocab)
        
        

In [10]:
vocab={"unk":0}

In [11]:
def build_vocab(texts):
  #print(texts["question"],texts["answer"])
  tokens_question=tokenize(texts["question"])
  tokens_answer=tokenize(texts["answer"])
  merged_texts=tokens_question+tokens_answer
  for text in merged_texts:
    if text not in vocab:
      vocab[text]=len(vocab)
  return vocab
  


In [12]:
df.apply(build_vocab,axis=1)

0     {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
1     {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
2     {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
3     {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
4     {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
                            ...                        
85    {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
86    {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
87    {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
88    {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
89    {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
Length: 90, dtype: object

In [13]:
vocab

{'unk': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 "'to": 12,
 'kill': 13,
 'a': 14,
 "mockingbird'": 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 "'1984'": 67,
 'george-orwell': 68,
 'currency': 69,
 'un

In [14]:
len(vocab)

326

In [15]:
## no of unique wirds is 91

In [16]:
def indices(texts,vocab):
    indices_list=[]
    tokens=tokenize(texts)
    for tok in tokens:
        if tok in vocab:
            indices_list.append(vocab[tok])
        else:
            indices_list.append(vocab["unk"])
    return indices_list
            

In [17]:
df["answer"][0]

'Paris'

In [18]:
indices("Paris",vocab)

[7]

In [19]:
## now we can convert all questions and answer to indices using dataset and dataloader 

In [20]:
from torch.utils.data import Dataset,DataLoader

In [21]:
import torch

In [22]:
class qadataset(Dataset):
    def __init__(self,df,vocab):
        self.df=df
        self.vocab=vocab
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        
        row=self.df.iloc[idx]
        question_indices=indices(row["question"],self.vocab)
        answer_indices=indices(row["answer"],self.vocab)
        return torch.tensor(question_indices),torch.tensor(answer_indices)
        

In [23]:
qdataset=qadataset(df,vocab)

In [24]:
qdataset[0]

(tensor([1, 2, 3, 4, 5, 6]), tensor([7]))

In [25]:
dataloader=DataLoader(qdataset,batch_size=1,shuffle=True)

In [26]:
len(dataloader)

90

In [27]:
for q,a in dataloader:
  print(q,a)

tensor([[ 42, 117, 118,   3, 119,  94, 120]]) tensor([[121]])
tensor([[ 1,  2,  3, 69,  5,  3, 70, 71]]) tensor([[72]])
tensor([[42, 86, 87, 88, 89, 39, 90]]) tensor([[91]])
tensor([[  1,   2,   3,   4,   5, 113]]) tensor([[114]])
tensor([[ 78,  79, 290,  81,  19,  14, 291]]) tensor([[85]])
tensor([[ 10,  11, 190, 159, 191]]) tensor([[192]])
tensor([[ 10,  11, 158, 159, 160]]) tensor([[161]])
tensor([[ 10, 140,   3, 141, 272,  93, 273,   5,   3, 274]]) tensor([[275]])
tensor([[  1,  87, 230, 231, 232, 233]]) tensor([[234]])
tensor([[  1,   2,   3,   4,   5, 288]]) tensor([[289]])
tensor([[ 42, 320,   2,  62,  63,   3, 321,   5, 322]]) tensor([[323]])
tensor([[ 1,  2,  3, 59, 25,  5, 26, 19, 60]]) tensor([[61]])
tensor([[ 42, 101,   2,   3,  17]]) tensor([[102]])
tensor([[ 1,  2,  3,  4,  5, 53]]) tensor([[54]])
tensor([[ 42, 257,   2, 258,  83, 259, 260]]) tensor([[261]])
tensor([[ 1,  2,  3,  4,  5, 73]]) tensor([[74]])
tensor([[  1,   2,   3, 181, 182, 183, 184]]) tensor([[185]])
ten

In [28]:
import torch.nn as nn

In [68]:
## architecture of model 
class simplernn(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.embedding=nn.Embedding(vocab_size,embedding_dim=50)
    self.rnn=nn.RNN(50,64)
    self.linear=nn.Linear(64,vocab_size)
  def forward(self,question):
    embedding_ques=self.embedding(question)
    hidden,final=self.rnn(embedding_ques)
    output=self.linear(final)
    return output


In [30]:
## here we understand why not we use sequential container 
##we give one dataset in this architectutre to understandthis 

In [31]:
qdataset[0]

(tensor([1, 2, 3, 4, 5, 6]), tensor([7]))

In [34]:
x=nn.Embedding(324,embedding_dim=50)

In [35]:
x(qdataset[0][0])## first index of first data question wala sirf 

tensor([[-5.2235e-01, -2.0357e+00,  5.0205e-01,  7.4209e-01, -1.2622e+00,
         -9.2104e-01,  6.1955e-02,  1.1126e-02, -8.0925e-01, -1.4837e+00,
          8.0193e-01,  7.0112e-01,  1.7865e+00,  9.8911e-01, -2.7709e-01,
         -5.7232e-01,  1.8588e+00,  1.1363e+00,  6.7856e-01,  7.2295e-01,
          7.7040e-01, -9.4850e-01,  1.4637e+00,  3.5896e-01,  1.0940e+00,
          5.5446e-01,  1.1671e+00,  6.9778e-01, -1.0611e+00, -1.9507e+00,
         -6.8580e-01, -2.9107e-01,  1.2794e+00,  1.5937e-01,  2.5960e-01,
         -1.1038e+00, -6.0730e-04, -9.2208e-01, -2.1200e-01,  3.8824e-01,
         -1.0668e+00, -8.9306e-01, -2.7714e-01,  1.8020e+00, -8.7957e-01,
         -8.4197e-01, -1.7537e-01, -8.8507e-01, -1.8276e+00,  2.5866e-01],
        [ 3.8951e-01,  4.7131e-01,  1.8359e+00,  4.8097e-01, -1.5245e+00,
         -2.2455e+00,  2.1290e+00,  1.1001e+00,  1.6257e-01,  7.9719e-01,
         -3.3696e-01, -4.7018e-01,  2.0588e+00, -1.2026e+00, -6.0380e-01,
         -2.4612e-01,  1.0034e+00, -8

## output main hume 50 dimension ka vector mila
## shape of this tensor 

In [37]:
qdataset[0][0].shape

torch.Size([6])

In [56]:
a=x(qdataset[0][0])

In [ ]:
 ## 6 vectorhain sabka size 50 dimension hain 

In [ ]:
## lets take 10 question 

In [38]:
x(qdataset[10][0])

tensor([[-5.2235e-01, -2.0357e+00,  5.0205e-01,  7.4209e-01, -1.2622e+00,
         -9.2104e-01,  6.1955e-02,  1.1126e-02, -8.0925e-01, -1.4837e+00,
          8.0193e-01,  7.0112e-01,  1.7865e+00,  9.8911e-01, -2.7709e-01,
         -5.7232e-01,  1.8588e+00,  1.1363e+00,  6.7856e-01,  7.2295e-01,
          7.7040e-01, -9.4850e-01,  1.4637e+00,  3.5896e-01,  1.0940e+00,
          5.5446e-01,  1.1671e+00,  6.9778e-01, -1.0611e+00, -1.9507e+00,
         -6.8580e-01, -2.9107e-01,  1.2794e+00,  1.5937e-01,  2.5960e-01,
         -1.1038e+00, -6.0730e-04, -9.2208e-01, -2.1200e-01,  3.8824e-01,
         -1.0668e+00, -8.9306e-01, -2.7714e-01,  1.8020e+00, -8.7957e-01,
         -8.4197e-01, -1.7537e-01, -8.8507e-01, -1.8276e+00,  2.5866e-01],
        [ 3.8951e-01,  4.7131e-01,  1.8359e+00,  4.8097e-01, -1.5245e+00,
         -2.2455e+00,  2.1290e+00,  1.1001e+00,  1.6257e-01,  7.9719e-01,
         -3.3696e-01, -4.7018e-01,  2.0588e+00, -1.2026e+00, -6.0380e-01,
         -2.4612e-01,  1.0034e+00, -8

In [ ]:
qdataset[15][0]

In [45]:
(qdataset[15][0]).shape

torch.Size([8])

In [51]:
aa=x(qdataset[15][0]).shape

## now make second layer 

In [48]:
y=nn.RNN(50,64)

In [57]:
y(a)

(tensor([[ 0.0176,  0.4574,  0.4549,  0.4541, -0.2827, -0.3343, -0.5861,  0.4408,
          -0.5464,  0.2845,  0.1246,  0.6965,  0.2751, -0.1463,  0.4017,  0.0883,
          -0.5314, -0.2409, -0.3146,  0.1483, -0.4961,  0.1818,  0.4294, -0.5238,
           0.6563,  0.4745,  0.0891, -0.8361, -0.7406,  0.7302, -0.2980,  0.0166,
          -0.1575, -0.3460,  0.1014, -0.0242, -0.4519,  0.1658,  0.1753, -0.4783,
          -0.6698,  0.0094, -0.3690, -0.3804,  0.6732, -0.0018,  0.3787,  0.3191,
           0.6532,  0.8159,  0.3892,  0.0611, -0.3389, -0.5810,  0.1731, -0.3206,
           0.5865,  0.3680, -0.0547, -0.2752, -0.5820, -0.2380,  0.2321, -0.1458],
         [-0.0426,  0.2887,  0.4500, -0.7256,  0.1886, -0.6070, -0.8936,  0.6004,
          -0.2249, -0.7061,  0.0912, -0.8310, -0.8581, -0.4959,  0.5631, -0.3259,
           0.6759,  0.7839, -0.3612,  0.2098, -0.6843,  0.7442,  0.0733, -0.7950,
           0.6671,  0.1854, -0.3708, -0.8462, -0.5128, -0.3474,  0.1260,  0.1202,
          -0.49

## yaha par do vector milte hain 

In [ ]:
## rnn hiddens state 
y(a)[0].shape

torch.Size([6, 64])

6 vector 64 dimension

In [65]:
b=y(a)[1]

1 vector 64 dimenson

## thats why we don t use sequential container nn .rnn se do output nikal rahe hain 

In [62]:
z=nn.Linear(64,324)

In [67]:
z(b).shape

torch.Size([1, 324])